In [2]:
import sys
# # sys.setrecursionlimit(10000)
import os
sys.path.append( os.getcwd() )
os.chdir('..')

In [5]:
from time import time
import numpy as np
from Tree.tree_builder import tree_builder
from Tree.tree_values import TreeValues
# from Tree.strategy_filling import strategy_filling
from Tree.tree_visualizer import tree_visualizer
from Tree.tree_cfr import TreeCFR
from helper_classes import TreeParams, Node
from Game.card_to_string_conversion import card_to_string
from Game.card_tools import card_tools
from Settings.constants import constants
from Settings.arguments import arguments
# from Nn.value_nn import ValueNn
# from Nn.next_round_value import NextRoundValue
# from Nn.mock_nn_terminal import MockNnTerminal
from TerminalEquity.terminal_equity import TerminalEquity
# from Training.train import Train
# from Tree.tree_strategy_filling import TreeStrategyFilling
from TerminalEquity.evaluator import evaluator
from Lookahead.resolving import Resolving
from DataGeneration.range_generator import RangeGenerator

In [6]:
from tqdm import tqdm_notebook as tqdm
# import plotly as py
# import plotly.graph_objs as go
%load_ext line_profiler

In [7]:
PC = constants.players_count
CC = constants.card_count
HC = constants.hand_count
P1 = constants.players.P1
P2 = constants.players.P2

# turn lookahead

In [4]:
# board = card_to_string.string_to_board('3c5h4h3h')
# a = card_tools.get_last_round_boards(board) + 1
# a

In [17]:
import tensorflow as tf
config = tf.ConfigProto()
config.graph_options.optimizer_options.global_jit_level = tf.OptimizerOptions.ON_1
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

In [8]:
res = []
batch_size = 2
ranges = np.load('./test_ranges/turn_nl.npy')
player_range = ranges[0].reshape([1,-1])
opponent_range = ranges[1].reshape([1,-1])

current_node = Node()
current_node.board = card_to_string.string_to_board('7d7c8s5s')
current_node.street = 3
current_node.current_player = constants.players.P2
current_node.bets = np.array([10000, 10000], dtype=arguments.dtype)
current_node.num_bets = 0

te = TerminalEquity()
te.set_board(current_node.board)

range_generator = RangeGenerator()
range_generator.set_board(te, current_node.board)

player_range = np.zeros([batch_size,1326])
range_generator.generate_range(player_range)
opponent_range = np.zeros([batch_size,1326])
range_generator.generate_range(opponent_range)

resolving = Resolving(te, verbose=1)

time = resolving.resolve(current_node, player_range, opponent_range)
res.append(time)

{-1: 0, 20000: 1}
Build time: 5.800560712814331


100%|██████████████████████████████████████████████████████████████| 1000/1000 [00:28<00:00, 35.21it/s]


times: [ 0.259  0.291  0.196  0.017 27.941  0.126  0.039  0.016]


C:\Users\domin\Documents\GitHub\PyStack\src\Lookahead\lookahead.py:427: RuntimeWarning: invalid value encountered in true_divide
  out.children_cfvs = out.children_cfvs / scaler


Resolve time: 28.917906522750854
printing batch: 0
root_cfvs - (2, 1326)
[-2188.86 -4152.32   397.73 13153.78 11962.6  11651.33]
root_cfvs_both_players - (2, 2, 1326)
[-2188.86 -4152.32   397.73 13153.78 11962.6  11651.33]
[ 1515.71  1467.76  4267.47 12707.89 11916.99 12666.85]
achieved_cfvs - (2, 2, 1326)
[ 1515.71  1467.76  4267.47 12707.89 11916.99 12666.85]
[-2188.86 -4152.32   397.73 13153.78 11962.6  11651.33]
strategy - (3, 2, 1326)
[[0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0.]]


In [8]:
print(res)
asd()

[136.790625333786, 258.0213177204132, 511.7652814388275, 998.3410956859589]


NameError: name 'asd' is not defined

In [ ]:
ranges = np.load('test_ranges/turn_nl.npy')
player_range = ranges[0].reshape([1,-1])
opponent_range = ranges[1].reshape([1,-1])

batch_size = 1
player_range = np.tile(player_range, batch_size).reshape([batch_size,-1])
opponent_range = np.tile(opponent_range, batch_size).reshape([batch_size,-1])

current_node = Node()
current_node.board = card_to_string.string_to_board('7d7c8s5s')
current_node.street = 3
current_node.current_player = constants.players.P2
current_node.bets = np.array([8000, 8000], dtype=arguments.dtype)
current_node.num_bets = 0

te = TerminalEquity()
te.set_board(current_node.board)

range_generator = RangeGenerator()
range_generator.set_board(te, current_node.board)

player_range = np.zeros([1,1326])
range_generator.generate_range(player_range)
opponent_range = np.zeros([1,1326])
range_generator.generate_range(opponent_range)

resolving = Resolving(te, verbose=1)

turn_results = resolving.resolve(current_node, player_range, opponent_range)

# %lprun -f resolving.lookahead._compute_current_strategies resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_ranges resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_terminal_equities_terminal_equity resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead.next_street_boxes.get_value resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_cfvs resolving.resolve(current_node, player_range, opponent_range)
# %lprun -f resolving.lookahead._compute_regrets resolving.resolve(current_node, player_range, opponent_range) 

In [ ]:
asd()

In [ ]:
v = turn_results.root_cfvs
m1 = np.ones_like(v)
m1[v == 0] = 0
m1.sum()

In [ ]:
v = opponent_range
m = np.ones_like(v)
m[v == 0] = 0
m.sum()
np.array_equal(m1,m)

In [ ]:
resolving.lookahead_tree.actions

In [ ]:
results = turn_results
print('3c 5h 4h 3h')
i = 0
for card1 in range(CC):
    for card2 in range(card1+1,CC):
        idx = card_tools.get_hole_index([card1,card2])
        if player_range[0,idx] > 0.0:
            i += 1
            if 300 < i and i < 360:
                print(card_to_string.card_to_string(card1), card_to_string.card_to_string(card2), end='')
                print(np.array2string(results.strategy[:,0,idx], suppress_small=True, precision=2), end='\t')
                print(results.root_cfvs[0,idx], end='\t')
                print(player_range[0,idx])
print(i)

In [ ]:
i_ = [f.path for f in os.scandir(os.path.join(arguments.data_path, 'river', 'npy')) if 'inputs' in f.path]
t_ = [f.path for f in os.scandir(os.path.join(arguments.data_path, 'river', 'npy')) if 'targets' in f.path]
b_ = [f.path for f in os.scandir(os.path.join(arguments.data_path, 'river', 'npy')) if 'board' in f.path]

In [ ]:
for j in range(10):
    print(j)
    print(np.load(b_[j])[0])
    i = np.load(i_[j])
    i = i[0,:-1]
    for card1 in range(CC):
        for card2 in range(card1+1,CC):
            idx = card_tools.get_hole_index([card1,card2])
            if i[idx] == 0:
                s = 'Ts'
                if s in card_to_string.card_to_string(card1) or s in card_to_string.card_to_string(card2):
                    print(card_to_string.card_to_string(card1), card_to_string.card_to_string(card2), card1, card2)

# river lookahead

In [10]:
np.random.choice(52, size=5, replace=False)

array([22, 10, 30, 48, 13])

In [9]:
ranges = np.load('test_ranges/river_nl.npy')
player_range = ranges[0].reshape([1,-1])
opponent_range = ranges[1].reshape([1,-1])

batch_size = 1
player_range = np.tile(player_range, batch_size).reshape([batch_size,-1])
opponent_range = np.tile(opponent_range, batch_size).reshape([batch_size,-1])

current_node = Node()
current_node.board = card_to_string.string_to_board('7d7c8s5sQd')
current_node.street = 4
current_node.current_player = constants.players.P2
current_node.bets = np.array([8000, 8000], dtype=arguments.dtype)
current_node.num_bets = 0

te = TerminalEquity()
te.set_board(current_node.board)

# range_generator = RangeGenerator()
# range_generator.set_board(te, current_node.board)

# player_range = np.zeros([1,1326])
# range_generator.generate_range(player_range)
# opponent_range = np.zeros([1,1326])
# range_generator.generate_range(opponent_range)

resolving = Resolving(te, verbose=1)

river_results = resolving.resolve(current_node, player_range, opponent_range) 

# %lprun -f resolving.lookahead._compute_current_strategies resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_ranges resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_terminal_equities_terminal_equity resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_terminal_equities_next_street_box resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_cfvs resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_regrets resolving.resolve(current_node, player_range, opponent_range) 

Build time: 0.0


100%|█████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 394.18it/s]


times: [0.001 0.197 0.121 0.016 1.815 0.212 0.16  0.016]
Resolve time: 2.5497653484344482
printing batch: 0
root_cfvs - (1, 1326)
[-4212.13 -4322.69 -4426.17 -1711.65 -1711.65 -1701.19]
root_cfvs_both_players - (1, 2, 1326)
[-4212.13 -4322.69 -4426.17 -1711.65 -1711.65 -1701.19]
[-7416.12 -7370.56 -7378.47 -6971.93 -6971.93 -6835.54]
achieved_cfvs - (1, 2, 1326)
[-7416.12 -7370.56 -7378.47 -6971.93 -6971.93 -6835.54]
[-4212.13 -4322.69 -4426.17 -1711.65 -1711.65 -1701.19]
strategy - (3, 1, 1326)
[[0.   0.   0.   0.   0.   0.  ]
 [0.02 0.02 0.02 0.99 0.99 0.99]
 [0.98 0.98 0.98 0.01 0.01 0.01]]


In [7]:
river_results.root_cfvs_both_players.max()

16013.057

In [ ]:
resolving.lookahead_tree.actions

In [ ]:
results = river_results
print('7d 7c 8s 5s Qd')
i = 0
for card1 in range(CC):
    for card2 in range(card1+1,CC):
        idx = card_tools.get_hole_index([card1,card2])
        if opponent_range[0,idx] > 0:
            i += 1
#             if 200 < i and i < 270:
            if '9s' in card_to_string.card_to_string(card2) or '9s' in card_to_string.card_to_string(card1):
                print(card_to_string.card_to_string(card1), card_to_string.card_to_string(card2), end='')
                print(np.array2string(results.strategy[:,0,idx], suppress_small=True, precision=2), end=' ')
                print(np.array2string(results.root_cfvs_both_players[0,0,idx], suppress_small=True, precision=2))
print(i)

# Board to nn feature

In [ ]:
board = card_to_string.string_to_board('7d7cKsKhQs')

te = TerminalEquity()
te.set_board(board)
range_generator = RangeGenerator()
range_generator.set_board(te, board)

r = np.zeros([1,1326])
range_generator.generate_range(r)

In [ ]:
m = np.zeros([1,1326])
m[r > 0.00001] = 1
print(m.sum())

In [ ]:
board = card_to_string.string_to_board('7c5c6c2c')
board2 = card_to_string.string_to_board('2hKcJdQc')
boards = np.zeros([2,board.shape[0]], dtype=int)
boards[0] = board
boards[1] = board2

In [ ]:
c = 52
p = 2 * 1
left_cards = c - 4 - p
print(left_cards)
tools.choose(left_cards, 1)

In [ ]:
card_tools.get_possible_hand_indexes(boards)

In [ ]:
a = evaluator.batch_eval_fast(boards)
b = evaluator.batch_eval_fast2(boards)
np.array_equal(a,b)

In [ ]:
%timeit a = evaluator.batch_eval_fast(boards)
%timeit b = evaluator.batch_eval_fast2(boards)

In [ ]:
_idx_to_cards = np.zeros([HC,2], dtype=arguments.dtype)
for card1 in range(CC):
    for card2 in range(card1+1,CC):
        idx = card_tools.get_hole_index([card1,card2])
        _idx_to_cards[idx][0] = card1
        _idx_to_cards[idx][1] = card2

def f():
    hands = np.zeros([HC, board.shape[0] + 2], dtype=arguments.int_dtype) # ? - long
    hands[ : ,  :board.shape[0] ] = board.reshape([1,board.shape[0]]) * np.ones([HC,board.shape[0]])
    hands[ : , -2: ] = _idx_to_cards.copy()
    a = evaluator.evaluate_fast(hands)

def f2():
    hands = np.zeros([HC, board.shape[0] + 2], dtype=arguments.int_dtype) # ? - long
    hands[ : ,  :board.shape[0] ] = board.reshape([1,board.shape[0]]) * np.ones([HC,board.shape[0]])
    hands[ : , -2: ] = _idx_to_cards.copy()
    b = card_tools.get_possible_hand_indexes(board).astype(bool)

In [ ]:
%timeit f()

In [ ]:
%timeit f2()

In [ ]:
_idx_to_cards[1300:]

In [ ]:
card_tools.get_possible_hand_indexes()

In [ ]:
board = card_to_string.string_to_board('7c5c6cQd')
print(board)
boards  = card_tools.get_next_round_boards(board) # []
mask = np.zeros([boards.shape[0], HC], dtype=bool)
for i, b in enumerate(boards):
    mask[i] = card_tools.get_possible_hand_indexes(b)

print(mask.shape)
print(boards.shape)
print(np.zeros([boards.shape[0], HC])[mask])

In [ ]:
card_tools.get_possible_hands_mask(np.array([[1,2,3,4,5,6,7]]))

In [ ]:
for i, j in enumerate(range(2,15)):
    print(i, j)

In [ ]:
game_settings.rank_count, game_settings.suit_count

In [ ]:
def convert_board_to_nn_feature(board):
    num_cards = game_settings.card_count
    one_hot_cards = np.zeros([num_cards], dtype=np.float32)
    for card in board:
        one_hot_cards[ card ] = 1
    return one_hot_cards
board = card_to_string.string_to_board('7d7c8s5hQd')
convert_board_to_nn_feature(board)

In [ ]:
def convert_board_to_nn_feature(board):
    num_ranks, num_suits = game_settings.rank_count, game_settings.suit_count
    num_cards = game_settings.card_count
    one_hot_board = np.zeros([num_cards], dtype=np.float32)
    suit_counts = np.zeros([num_suits], dtype=np.float32)
    rank_counts = np.zeros([num_ranks], dtype=np.float32)
    for card in board:
        suit = card_to_string.card_to_suit(card) 
        rank = card_to_string.card_to_rank(card)
        one_hot_board[ card ] = 1
        suit_counts[ suit ] += 1
        rank_counts[ rank ] += 1
    # normalize counts
    rank_counts /= num_ranks
    suit_counts /= num_suits
    # combine all arrays and return
    out = np.zeros([num_cards + num_suits + num_ranks], dtype=np.float32)
    out[ :num_cards ] = one_hot_board
    out[ num_cards:num_cards+num_suits ] = suit_counts
    out[ num_cards+num_suits: ] = rank_counts
    return out
board = card_to_string.string_to_board('7c7c8s5hQd')
convert_board_to_nn_feature(board)

# Range Generator

In [ ]:
board = card_to_string.string_to_board('7d7c8s5sQd')
range_generator = RangeGenerator()
term_eq = TerminalEquity()
term_eq.set_board(board)
range_generator.set_board(term_eq, board)

In [ ]:
batch_size = 16
ranges = np.zeros([batch_size, HC], dtype=arguments.dtype)
range_generator.generate_range(ranges)

# Random functions

In [ ]:
# card_to_string.string_to_board('KsKhJs')

In [ ]:
# card_tools.string_to_hole_index('Jh')

In [ ]:
# a = card_tools.get_next_round_boards(np.zeros([]))
# a[-5:]

In [ ]:
# card_tools.get_last_boards_count(1)

In [ ]:
# card_tools.get_flop_board_index(np.array([10,20,30])-1)

In [ ]:
# card_tools.get_hole_index([11,14])

In [ ]:
# card_tools.get_board_index(np.array([2,20,30,40,10])-1)

In [ ]:
# evaluator.evaluate_seven_card_hand(np.array([2,4,8,16,32,15,7])-1)

In [ ]:
# a = np.arange(6*7).reshape([6,7])
# evaluator.evaluate_fast(a)

In [ ]:
# evaluator.batch_eval_fast(np.array([0,1,2]))[783]

# Tree Build

In [ ]:
# params = TreeParams()
# first_node = Node()
# first_node.board = np.zeros([], dtype=int)
# first_node.street = 1
# first_node.current_player = constants.players.P1
# first_node.bets = np.array([50, 100], dtype=int)
# first_node.num_bets = 0
# params.root_node = first_node
# params.limit_to_street = True

# current_node = Node()
# current_node.board = card_to_string.string_to_board('7d7c8s5sQd')
# current_node.street = 4
# current_node.current_player = constants.players.P2
# current_node.bets = np.array([100, 100], dtype=arguments.dtype)
# current_node.num_bets = 0
# params.root_node = current_node

# root = tree_builder.build_tree(params)

In [ ]:
# tree_visualizer.draw_tree(root, size='30,10', show_vars=True)

# CFR

In [ ]:
# card_to_string.string_to_board('7d7c8s5sQd')

In [ ]:
def _generate_sorted_range(ranges):
    batch_size = ranges.shape[0]
    mass = np.ones([batch_size], dtype=np.float32)
    _generate_recursion(ranges, mass)

def _generate_recursion(cards, mass):
    batch_size = cards.shape[0]
    assert(mass.shape[0] == batch_size)
    card_count = cards.shape[1]
    # we terminate recursion at size of 1
    if card_count == 1:
        cards[ : , 0 ] = mass.copy()
    else:
        rand = np.random.rand(batch_size)
        mass1 = mass.copy() * rand
        mass2 = mass - mass1
        halfSize = card_count / 2
        # if the tensor contains an odd number of cards,
        # randomize which way the middle card goes
        if halfSize % 1 != 0:
            # if end is .5 then init randomly between two numbers
            halfSize = int(halfSize - 0.5)
            halfSize = halfSize + np.random.randint(2) # (0 or 1)
        halfSize = int(halfSize)

        _generate_recursion(cards[ : , :halfSize ], mass1)
        _generate_recursion(cards[ : , halfSize: ], mass2)

In [ ]:
# ranges = np.load('test_ranges/river_nl.npy')
# player_range = ranges[0].reshape([1,-1])
# opponent_range = ranges[1].reshape([1,-1])

# def test(iters=np.linspace(100,2000,10, dtype=int), skip=arguments.cfr_skip_iters, kfold=5):
#     kf = []
#     for k in tqdm(range(kfold)):
#         results = []
#         for i in tqdm(iters, total=len(iters)):
#             if i >= skip:
#                 params = TreeParams()

#                 ranges = np.zeros([1,1326])
#                 _generate_sorted_range(ranges)
#                 player_range = ranges.reshape([-1])
#                 ranges = np.zeros([1,1326])
#                 _generate_sorted_range(ranges)
#                 opponent_range = ranges.reshape([-1])

#                 params.root_node = Node()
#                 params.root_node.board = card_generator.generate_cards(5) # card_to_string.string_to_board('6c6d2c2d3h')
#                 params.root_node.street = 4
#                 params.root_node.current_player = constants.players.P1 if np.random.rand() > 0.5 else constants.players.P2
#                 r = np.random.randint(1,20000)
#                 params.root_node.bets = np.array([r,r])
#                 params.root_node.num_bets = 0

#                 tree = tree_builder.build_tree(params)

#                 starting_ranges = np.zeros([PC,HC], dtype=arguments.dtype)
#                 starting_ranges[0] = player_range # card_tools.get_uniform_range(params.root_node.board)
#                 starting_ranges[1] = opponent_range # card_tools.get_uniform_range(params.root_node.board)

#                 t0 = time()
#                 tree_cfr = TreeCFR()
#                 # %lprun -f tree_cfr.cfrs_iter_dfs tree_cfr.run_cfr(tree, starting_ranges)
#                 tree_cfr.run_cfr(tree, starting_ranges, iter_count=i, skip=i-skip)
#                 print('Took time: {} for {} iters'.format(time()-t0, i))
#                 time_ = time()-t0

#                 tree_values = TreeValues()
#                 tree_values.compute_values(tree, starting_ranges)

#                 print('Exploitability: ' + str(tree.exploitability) + ' [chips]' )
#                 results.append([i, time_, tree.exploitability])
#         kf.append(np.array(results))
#     return np.array(kf)

In [ ]:
# iters = [700]
# skip = 300
# ats = test(iters, skip=skip, kfold=30)

In [ ]:
# ats.shape
# print(np.mean(ats, axis=0)[0,2], np.std(ats, axis=0)[0,2])

In [ ]:
# iters0 = np.linspace(0,1000,100, dtype=int)
# # iters1 = np.linspace(550,1000,10, dtype=int)
# iters1 = [700,800,900,1000]
# skip0 = 300
# skip1 = 300
# kfold = 100

In [ ]:
# ats = test(iters0, skip=skip0, kfold=kfold)
# x = ats[ : , : , 0]
# y1 = ats[ : , : , 1]
# y2 = ats[ : , : , 2]

In [ ]:
# ats = test(iters1, skip=skip1, kfold=kfold)
# _x = ats[ : , : , 0]
# _y1 = ats[ : , : , 1]
# _y2 = ats[ : , : , 2]

In [ ]:
# lambd = 0.999
# x = 1
# for i in range(1000):
#     x = lambd ** i
#     if i == 999:
#         print(x)

In [ ]:
# print(np.mean(x,axis=0))
# print(np.mean(y2,axis=0))
# print(np.std(y2,axis=0))

In [ ]:
# plot_2yaxis(x, y1, y2, _x, _y1, _y2, y_name='skip:{}'.format(skip0), comp_y_name='skip:{}'.format(skip1))
# # plot_2yaxis(_x, _y1, _y2, y_name='skip:{}'.format(skip0), comp_y_name='skip:{}'.format(skip1))

In [ ]:
# import matplotlib.pyplot as plt

# def plot_2yaxis(x, y1, y2, comp_x=None, comp_y1=None, comp_y2=None, y_name='', comp_y_name=''):
#     fig, ax1 = plt.subplots()
#     x = np.mean(x, axis=0)
#     y1 = np.mean(y1, axis=0)

#     ax1.plot(x, y1, 'b-')
#     ax1.set_xlabel('# of iterations')
#     ax1.set_ylabel('seconds', color='b')
#     ax1.tick_params('y', colors='b')

#     ax2 = ax1.twinx()
#     mean = np.mean(y2, axis=0)
#     std = np.mean(y2, axis=0)
#     ax2.plot(x, mean, lw = 3, color='r', alpha = 0.8, label = y_name)
#     y_low = mean - std
#     y_high = mean + std
#     ax2.fill_between(x, y_low, y_high, color='r', alpha = 0.1)
#     ax2.set_ylabel('exploitability', color='r')
#     ax2.tick_params('y', colors='r')

#     if comp_x is not None:
#         comp_x = np.mean(comp_x, axis=0)
#         comp_y1 = np.mean(comp_y1, axis=0)

#         ax1.plot(comp_x, comp_y1, 'b--')
#         ax1.tick_params('y', colors='b')

#         mean = np.mean(comp_y2, axis=0)
#         std = np.mean(comp_y2, axis=0)
#         ax2.plot(comp_x, mean, 'r--', lw = 3, alpha = 0.8, label = comp_y_name)
#         y_low = mean - std
#         y_high = mean + std
#         ax2.fill_between(comp_x, y_low, y_high, color='g', alpha = 0.1)
#         ax2.tick_params('y', colors='r')
        
# #         ax.grid()
#     ax2.legend()
#     fig.tight_layout()
#     plt.show()

In [ ]:
# print(np.array2string(tree.strategy[:, 522:527], suppress_small=True, precision=5))

# Flop nl

In [ ]:
%load_ext line_profiler

In [ ]:
ranges = np.load('test_ranges/river_nl.npy')
player_range = ranges[0].reshape([1,-1])
opponent_range = ranges[1].reshape([1,-1])

batch_size = 16
player_range = np.tile(player_range, batch_size).reshape([batch_size,-1])
opponent_range = np.tile(opponent_range, batch_size).reshape([batch_size,-1])

current_node = Node()
current_node.board = card_to_string.string_to_board('7d7c8s5sQd')
current_node.street = 4
current_node.current_player = constants.players.P2
current_node.bets = np.array([8000, 8000], dtype=arguments.dtype)
current_node.num_bets = 0

te = TerminalEquity()
te.set_board(current_node.board)

resolving = Resolving(te, verbose=1)

results = resolving.resolve(current_node, player_range, opponent_range) 

# %lprun -f resolving.lookahead._compute_current_strategies resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_ranges resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_terminal_equities_terminal_equity resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_terminal_equities_next_street_box resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_cfvs resolving.resolve(current_node, player_range, opponent_range) 
# %lprun -f resolving.lookahead._compute_regrets resolving.resolve(current_node, player_range, opponent_range) 

In [ ]:
print('7d 7c 8s 5s Qd')
i = 0
for card1 in range(CC):
    for card2 in range(card1+1,CC):
        idx = card_tools.get_hole_index([card1,card2])
        if player_range[0,idx] > 0:
            i += 1
            if i < 10:
                print(card_to_string.card_to_string(card1), card_to_string.card_to_string(card2), end='')
                print(np.array2string(results.strategy[:,0,idx], suppress_small=True, precision=2), end=' ')
                print(np.array2string(results.root_cfvs[0,idx], suppress_small=True, precision=2))
print(i)

In [ ]:
board = card_to_string.string_to_board('7d7c8s5sQd')
card_tools.get_possible_hand_indexes(board).sum()

In [ ]:
ranges = np.load('test_ranges/river_nl.npy')
print(ranges[0][1:100])
card_tools.get_possible_hand_indexes(board)[1:100]

In [ ]:
b = results.root_cfvs_both_players.copy().reshape([16,1326*2])[0]
m = np.zeros([1326*2])
m[ b == 0 ] = 1
m = m.reshape(1326*2)
a = np.arange(1326*2)
# print(a[ m > 0 ])

i = np.zeros([2,1326])
i[0] = player_range[0]
i[1] = opponent_range[0]
i = i.reshape([1326*2])
m2 = np.zeros([1326*2])
m2[ i > 0 ] = 1
print(m.sum())
print(m2.sum())
# print(a[ m2 > 0 ])

In [ ]:
# [-60.87 -61.67 -62.58 -21.3  -21.3  -21.89]
# [-94.51 -92.84 -92.76 -84.73 -84.73 -83.04]

In [ ]:
# [-7599.89 -7464.71 -7458.21 -7214.99 -7214.99 -7077.32]
# [-2017.01 -2134.09 -2233.93 -1670.9  -1670.9  -1681.84]

# (1, 1326) (1, 2, 1326)
# [-4212.14 -4322.69 -4426.17 -1711.65 -1711.65 -1701.19]
# [-4212.14 -4322.69 -4426.17 -1711.65 -1711.65 -1701.19]
# (2, 1326)
# [-7416.12 -7370.56 -7378.47 -6971.93 -6971.93 -6835.54]
# [-4212.14 -4322.69 -4426.17 -1711.65 -1711.65 -1701.19]

# [[0.45 0.45 0.45 0.   0.   0.  ]
#  [0.53 0.54 0.54 0.98 0.98 0.99]
#  [0.01 0.01 0.01 0.01 0.01 0.01]]

# [[0.   0.   0.   0.   0.   0.  ]
#  [0.02 0.02 0.02 0.99 0.99 0.99]
#  [0.98 0.98 0.98 0.01 0.01 0.01]]

# Batch size test

In [ ]:
# times = [5,16,20,30,40,50,100]
# r = []
# for _ in tqdm(range(3)):
#     res = []
#     for batch_size in tqdm(times, total=len(times)):
# #         I = 1326

# #         player_range = np.zeros([batch_size,I])
# #         _generate_sorted_range(player_range)
# #         opponent_range = np.zeros([batch_size,I])
# #         _generate_sorted_range(opponent_range)
#         ranges = np.load('test_ranges/river_nl.npy')
#         player_range = ranges[0].reshape([1,-1])
#         opponent_range = ranges[1].reshape([1,-1])

#         player_range = np.tile(player_range, batch_size).reshape([batch_size,-1])
#         opponent_range = np.tile(opponent_range, batch_size).reshape([batch_size,-1])


#         current_node = Node()
#         current_node.board = card_to_string.string_to_board('7d7c8s5sQd')
#         current_node.street = 4
#         current_node.current_player = constants.players.P2
#         current_node.bets = np.array([100, 100], dtype=arguments.dtype)
#         current_node.num_bets = 0

#         te = TerminalEquity()
#         te.set_board(current_node.board)

#         resolving = Resolving(te, verbose=1)

#         results = resolving.resolve(current_node, player_range, opponent_range) 
#         res.append(results)
#     r.append(np.array(res))

In [ ]:
del r
ats = np.array(r)
ats.shape
ats /= times
print(np.mean(ats,axis=0))
print(np.std(ats,axis=0))

In [ ]:
# ats2 = ats.copy()
# ats = np.clip(ats,0,12)

In [ ]:
times = np.array([5,16,20,30,40,50,100])
ats = np.array([ [18,62,82,131,173,221,441], [16,62,81,128,170,217,442], [17,61,81,126,169,217,445] ])

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# x = times
# y = np.array(ats)
# mean = np.mean(y, axis=0)
# std = np.std(y, axis=0)
# print(mean, std)

# fig, ax = plt.subplots()

# ax.plot(x, mean, lw = 3, color='b', alpha = 0.8, label = '')
# y_low = mean - std
# y_high = mean + std
# ax.fill_between(x, y_low, y_high, color='b', alpha = 0.1)

# ax.set(xlabel='batches (nr)', ylabel='seconds (s)',
#        title='solving batches w/ same board')
# ax.grid()

In [ ]:
# import matplotlib.pyplot as plt

# x = times
# y = np.array(ats) / x
# mean = np.mean(y, axis=0)
# std = np.std(y, axis=0)

# fig, ax = plt.subplots()

# ax.plot(x, mean, lw = 3, color='b', alpha = 0.8, label = '')
# y_low = mean - std
# y_high = mean + std
# ax.fill_between(x, y_low, y_high, color='b', alpha = 0.1)

# ax.set(xlabel='batches (nr)', ylabel='seconds for 1 example (s)',
#        title='solving batches w/ same board')
# ax.grid()

# Card and Range Generator

In [ ]:
# from DataGeneration.random_card_generator import card_generator
# card_generator.generate_cards(6)

In [ ]:
# from DataGeneration.range_generator import RangeGenerator
# r = RangeGenerator()
# board = np.zeros([])
# r.set_board(board)
# BS,CC = 10,6
# ranges = np.zeros([BS,CC])
# r.generate_range(ranges)
# print(np.array2string(ranges, suppress_small=True, precision=2))
# print(np.array2string(np.sum(ranges, axis=1, keepdims=True), suppress_small=True, precision=2))

# Lookahead (test 1)

In [ ]:
# params = TreeParams()
# params.root_node = Node()
# params.root_node.board = card_to_string.string_to_board('Ks')
# params.root_node.street = 2
# params.root_node.current_player = constants.players.P1
# params.root_node.bets = np.array([100, 100])
# params.limit_to_street = False
# tree = tree_builder.build_tree(params)

In [ ]:
# from Tree.tree_strategy_filling import TreeStrategyFilling

# filling = TreeStrategyFilling()

# range1 = card_tools.get_uniform_range(params.root_node.board)
# range2 = card_tools.get_uniform_range(params.root_node.board)

# filling.fill_strategies(tree, P1, range1, range2)
# filling.fill_strategies(tree, P2, range1, range2)

In [ ]:
# starting_ranges = np.zeros([PC,CC], dtype=arguments.dtype)
# starting_ranges[0] = card_tools.get_uniform_range(params.root_node.board)
# starting_ranges[1] = card_tools.get_uniform_range(params.root_node.board)

# tree_values = TreeValues()
# tree_values.compute_values(tree, starting_ranges)

# print('Exploitability: ' + str(tree.exploitability) + ' [chips]' )

# Next Round Value Bucketing test

In [ ]:
# next_round_value = NextRoundValue(None)
# print(next_round_value._range_matrix)
# # test of card to bucket range translation
# range_ = np.arange(1,7).reshape([1,-1]).astype(float)
# next_round_range = np.zeros([1, next_round_value.bucket_count * next_round_value.board_count], dtype=arguments.dtype)
# next_round_value._card_range_to_bucket_range(range_, next_round_range)
# print(next_round_range)

# Next Round Value (mock test)

In [ ]:
# # # test of get_value functionality
# mock_nn = MockNnTerminal()
# # mock_nn = ValueNn()
# next_round_value = NextRoundValue(mock_nn)

# # bets = np.array([100], dtype=float)
# bets = np.array([1200], dtype=float)

# next_round_value.start_computation(bets)

# ranges = np.full([1,2,CC], 1/4, dtype=arguments.dtype)
# values = np.zeros([1,2,CC], dtype=arguments.dtype)

# np.random.seed(0)
# ranges[0][0] = np.array([1,1,0,0,0,0])
# ranges[0][1] = np.array([1,1,1,1,1,1])

# next_round_value.get_value(ranges, values)

# print(values)

# ranges_2 = ranges.reshape([2,CC]).copy()
# values_2 = np.full_like(ranges_2, -1)

# terminal_equity = TerminalEquity()
# terminal_equity.set_board(np.zeros([]))
# terminal_equity.call_value(ranges_2, values_2)
# print('terminal_equity')
# print(values_2)

# # output:
# # [[[ 2.  2.  0.  0. -2. -2.]
# #   [ 0.  0. -1. -1. -1. -1.]]]
# # terminal_equity
# # [[ 0.  0. -1. -1. -1. -1.]
# #  [ 2.  2.  0.  0. -2. -2.]]

In [ ]:
# # needs to comment out some asserts and use_memory = True

# board = card_to_string.string_to_board('Ks')

# values_3 = np.full_like(values, -1)
# next_round_value.get_value_on_board(board, values_3)

# print(values_3)

# # output:
# # [[[ 0.2  0.2  0.   0.8 -0.6 -0.6]
# #   [ 0.   0.   0.   1.  -1.  -1. ]]]

# Nn output

In [ ]:
# data_dir = os.path.join(arguments.data_path, 'tfrecords')
# T = Train(data_dir=data_dir)
# T.train(num_epochs=100, verbose=0)
# model = T.keras_model

In [ ]:
# npy_folder = r'C:\Users\domin\Documents\GitHub\PyStack\Data\TrainSamples\PotBet\npy'
# m = np.load(npy_folder + r'\masks.0.npy')
# x = np.load(npy_folder + r'\inputs.0.npy')
# y = np.load(npy_folder + r'\targets.0.npy')

# y_hat = model.predict(x)

# temp = m[0]
# m = np.zeros([72])
# m[:36] = temp
# m[36:] = temp
# x = x[0]
# y = y[0]
# y_hat = y_hat[0]

# d = np.dot(y_hat, x[:-1])
# d /= 2
# y_new = y_hat - d
# y_new = y_hat

# (y_new*m)[:36] - (y_new*m)[36:]

# Continual Re-solving in Tree

In [ ]:
# board = card_to_string.string_to_board('')

In [ ]:
# params = TreeParams()
# params.root_node = Node()
# params.root_node.board = board
# params.root_node.street = 1
# params.root_node.current_player = constants.players.P1
# params.root_node.bets = np.array([300, 300])
# params.limit_to_street = False
# tree = tree_builder.build_tree(params)

In [ ]:
# filling = TreeStrategyFilling()

# range1 = card_tools.get_uniform_range(board)
# range2 = card_tools.get_uniform_range(board)

# filling.fill_strategies(tree, P1, range1, range2)
# filling.fill_strategies(tree, P2, range1, range2)

In [ ]:
# starting_ranges = np.zeros([PC,CC], dtype=arguments.dtype)
# starting_ranges[0] = card_tools.get_uniform_range(params.root_node.board)
# starting_ranges[1] = card_tools.get_uniform_range(params.root_node.board)

# tree_values = TreeValues()
# tree_values.compute_values(tree, starting_ranges)

# print('Exploitability: ' + str(tree.exploitability) + ' [chips]' )

In [ ]:
# tree_visualizer.draw_tree(tree, size='20,10')

# Range generation

In [ ]:
# import plotly.plotly as py
# import plotly.graph_objs as go
# def plot(x, x2=None, num_buckets=1000, clip=10, title='', x_name='', x2_name=''):
#     bins = np.linspace(0, 1, num_buckets)
#     inds = np.digitize(x, bins)
#     bucketized_x = bins[ inds ]
#     bucketized_x_counts = np.zeros([I, num_buckets])
#     for i in range(bucketized_x.shape[1]):
#         x_, counts = np.unique(bucketized_x[:,i], return_counts=True)
#         bucketized_x_counts[i, :len(counts) ] = counts
# #     bucketized_x_counts /= x.shape[0]
#     bucketized_x_counts = np.transpose(bucketized_x_counts, [1,0])
#     bucketized_x_counts = bucketized_x_counts[ :clip , : ]
#     x_axis = bins
#     x_axis = x_axis[:clip]
    
#     data = []
    
#     data.append(go.Surface(
#                     z=bucketized_x_counts,
#                     y=x_axis,
#                     name=x_name
#                 ))
    
#     if x2 is not None:
#         inds = np.digitize(x2, bins)
#         bucketized_x = bins[ inds ]
#         bucketized_x_counts = np.zeros([I, num_buckets])
#         for i in range(bucketized_x.shape[1]):
#             x_, counts = np.unique(bucketized_x[:,i], return_counts=True)
#             bucketized_x_counts[i, :len(counts) ] = counts
# #         bucketized_x_counts /= x.shape[0]
#         bucketized_x_counts = np.transpose(bucketized_x_counts, [1,0])
#         bucketized_x_counts = bucketized_x_counts[ :clip , : ]
#         data.append(go.Surface(
#                     z=bucketized_x_counts,
#                     y=x_axis,
#                     name=x2_name
#                 ))
    
#     layout = go.Layout(
#         showlegend=True,
#         title=title,
#         autosize=False,
#         width=500,
#         height=500,
#         margin=dict(
#             l=65,
#             r=50,
#             b=65,
#             t=90
#         ),
#         xaxis=dict(
#             tickmode='linear',
#             ticks='outside',
#             tick0=0,
#             dtick=1/num_buckets,
#             ticklen=1,
# #             tickcolor='#000'
#         )
#     )
#     fig = go.Figure(data=data, layout=layout)
#     return py.iplot(fig, filename='elevations-3d-surface')

In [ ]:
# x = np.random.rand(batch_size,I)
# x = x / np.sum(x, axis=1).reshape([batch_size,1])
# plot(x, num_buckets=num_buckets, clip=clip)

In [ ]:
# I = 100
# batch_size = 1000
# ranges = np.zeros([batch_size,I])
# _generate_sorted_range(ranges)
# ranges
# x = ranges[ : , 0 ]
# y = np.arange(len(x))
# scatterplot(x,y)

In [ ]:
# x = np.random.rand(batch_size,I)
# x = x / np.sum(x, axis=1).reshape([batch_size,1])
# x = x[ : , 2 ]
# scatterplot(x,y)

In [ ]:
def scatterplot(x, y):
    trace = go.Scatter(
        x = x,
        y = y,
        mode = 'markers'
    )

    data = [trace]

    return py.iplot(data, filename='basic-scatter')

In [ ]:
def to_buckets2d(x, num_buckets, clip):
    bins = np.linspace(0, 1, num_buckets)
    inds = np.digitize(x, bins)
    bucketized_x = bins[ inds ]
    bucketized_x_counts = np.zeros([I, num_buckets])
    for i in range(bucketized_x.shape[1]):
        x_, counts = np.unique(bucketized_x[:,i], return_counts=True)
        bucketized_x_counts[i, :len(counts) ] = counts
    bucketized_x_counts /= batch_size
    bucketized_x_counts = np.mean(bucketized_x_counts, axis=0)
    bucketized_x_counts = bucketized_x_counts[ :clip ]
    bins = bins[ :clip ]
    return bucketized_x_counts, bins

In [ ]:
def plot2d(x1, x2, bins, x1_name='', x2_name=''):
    trace1 = go.Scatter(
        x = bins,
        y = x1,
        mode = 'lines',
        name = x1_name
    )
    trace2 = go.Scatter(
        x = bins,
        y = x2,
        mode = 'lines',
        name = x2_name
    )

    data = [trace1, trace2]
    return py.iplot(data, filename='scatter-mode')

In [ ]:
# I = 1300
# batch_size = 10000
# clip = 1000
# num_buckets = 10000
# ranges = np.zeros([batch_size,I])
# _generate_sorted_range(ranges)
# x = np.random.rand(batch_size,I)
# x = x / np.sum(x, axis=1).reshape([batch_size,1])
# x1, bins = to_buckets2d(x, num_buckets, clip)
# x2, bins = to_buckets2d(ranges, num_buckets, clip)
# plot2d(x1, x2, bins, x1_name='Random', x2_name='Recursive')

In [ ]:

# ranges = np.zeros([batch_size,I])
# _generate_sorted_range(ranges)
# x = np.random.rand(batch_size,I)
# x = x / np.sum(x, axis=1).reshape([batch_size,1])
# plot(x, x2=ranges, num_buckets=num_buckets, clip=clip, x2_name='Recursive', x_name='Random')

In [ ]:
# bucketized_x_counts = np.zeros([I, num_buckets])
# for i in range(bucketized_x.shape[1]):
#     x_, counts = np.unique(bucketized_x[:,i], return_counts=True)
#     bucketized_x_counts[i, :len(counts) ] = counts
# bucketized_x_counts /= el
# bucketized_x_counts = np.transpose(bucketized_x_counts, [1,0])

In [ ]:
# x_axis = bins

In [ ]:
# clip = 10
# bucketized_x_counts = bucketized_x_counts[ :clip , : ]
# x_axis = x_axis[:clip]

In [ ]:
# print(bucketized_x_counts.shape)
# print(x_axis.shape)